STAT 413
ESTIMATING SURFACE AREA OF COUNTRIES USING MONTE CARLO
-Aditya Chilwal
-Paul Bakshi 
-Muzammil Arshad

In [3]:
###### REQUIRED PACKAGES AND FUNCTIONS
install.packages("rworldmap")
install.packages("vMF")
install.packages("rgeos")

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



In [7]:
###### REQUIRED PACKAGES AND FUNCTIONS
library(rgeos)
library("vMF")
library(sp)
library(rworldmap)
data(countryExData)

# Tell whether a coordinate is within a country or not
coords2country <- function( latLong, country )
{  
  countriesSP <- getMap(resolution='low')
  
  #setting CRS directly to that from rworldmap
  pointsSP <- SpatialPoints(latLong, proj4string=CRS(proj4string(countriesSP)))  
  # print(pointsSP)
  # use 'over' to get indices of the Polygons object containing each point 
  indices <- over(pointsSP, countriesSP)
  
  # Check if coordinates are within country
  coords <- ((indices$ADMIN) == country)
  
  # Replace NA values with FALSE
  coords <- replace(coords, is.na(coords), FALSE)
  
}

# Get the actual size of a country in sq km
country_size <- function(country) 
{
  as.integer(countryExData[countryExData['Country'] == country][8])
    
}


# Conversion functions

deg2rad <- function(x) 
{
  x/180 * base::pi
}

rad2deg <- function(x) 
{
  x / base::pi * 180
}

xyz2latlon <- function(m) # altered it so it taken in matrix
{
  x <- m[,1]
  y <- m[,2]
  z <- m[,3]
  # rescale to unit sphere
  R <- sqrt( x^2 + y^2 + z^2)
  x <- x/R; y <- y/R; z <- z/R;
  
  r <- sqrt( x^2 + y^2)
  lat <- rad2deg( asin(z) )
  
  long <- (r>0) * rad2deg( acos(x/r) ) 
  long <- ( 1 - 2 * (y < 0) ) * long
  long [ is.na(long) ] <- 0
  
  return (data.frame(lon=c(long), lat=c(lat)))
}

# source:
# https://github.com/ProjectMOSAIC/mosaic/blob/master/R/rgeo.R


rsphere.uniform <- function( nn, kk=3 )
{
  xx = matrix(        # generate nn sets of normal variates 
    runif( nn*kk, -1, 1 ), nrow=nn, ncol=kk, byrow = T
  );
  
  rs = sqrt(rowSums( xx^2 ))  # Get squared rowsums for normalization
  pp = xx/rs  # note that R applies operations like / elementwise down each column
  return( pp )
}

confidence_interval <- function( real_value, est, vari, nn, alf )
{
  conf_interval = c(est - qnorm(1-alf/2)*sqrt(vari/(nn-1)), est + qnorm(1-alf/2)*sqrt(vari/(nn-1)) )
  
}


## Random point on a sphere from a normal variate ##
rsphere.normal <- function( nn, kk=3 ){
  xx = matrix(        # generate nn sets of normal variates 
    rnorm( nn*kk, 0, 1 ), nrow=nn, ncol=kk, byrow = T
  );
  rs = sqrt(rowSums( xx^2 ))  # Get squared rowsums for normalization
  pp = xx/rs  # note that R applies operations like / elementwise down each column
  return( pp )
}

rsphere.importance_uniform <- function( nn, kk=3 )
{
  xx = matrix(        # generate nn sets of normal variates 
    runif( nn*kk, -1, 1 ), nrow=nn, ncol=kk, byrow = T
  );
  
  rs = sqrt(rowSums( xx^2 ))  # Get squared rowsums for normalization
  pp = xx/rs  # note that R applies operations like / elementwise down each column
  
  in_top_half <- pp[, 3] >= 0
  pp <- pp[in_top_half, ]  
  
  return( pp )
}

## Wrap around version
coords2country_wrap <- function( long_lat, country )
{  
  
  # wrap around longitude
  long_lat[,1] <- ifelse(long_lat[,1] > 180, long_lat[,1] - 360, long_lat[,1])
  long_lat[,1] <- ifelse(long_lat[,1] <= -180, long_lat[,1] + 360, long_lat[,1])
  # wrap around latitude
  long_lat[,2] <-  ifelse(long_lat[,2] > 90, long_lat[,2] - 180, long_lat[,2])
  long_lat[,2] <-  ifelse(long_lat[,2] <= -90, long_lat[,2] + 180, long_lat[,2])
  
  countriesSP <- getMap(resolution='low')
  
  #setting CRS directly to that from rworldmap
  pointsSP <- SpatialPoints(long_lat, proj4string=CRS(proj4string(countriesSP)))  
  # print(pointsSP)
  # use 'over' to get indices of the Polygons object containing each point 
  indices <- over(pointsSP, countriesSP)
  
  # Check if coordinates are within country
  coords <- ((indices$ADMIN) == country)
  
  # Replace NA values with FALSE
  coords <- replace(coords, is.na(coords), FALSE)
}

# density for rescaled beta
#r is the points we want the density value for
# a,b are alpha and beta parameters respectively
# l is lower bound
# u is upper bound
density_beta= function(r,a,b,l,u){
  density_val = (1/(u-l))*dbeta((r-l)/(u-l),a,b)
  return (density_val)
}


country_center <- function(country)
{
    # returns in long lat format
    countriesSP <- getMap(resolution='low')
    
    centroids <- gCentroid(countriesSP, byid=TRUE)
    
    df <- as.data.frame(centroids)
    
    df[country,]
}
##############################
# RELEVANT PRINT STATEMENTS PRINTER
###############################
print_statements<-function (n, alf, country_name, scaled_mean, scaled_var){
    true_size <- country_size(country_name)
    print(paste('Country: ', country_name))
    print(paste("Mean: ",scaled_mean))
    print(paste("Variance: ", scaled_var))


    range <- confidence_interval(true_size, scaled_mean, scaled_var, n, alf)
    inside <- (range[1] <= true_size && range[2] >= true_size)


    print(paste((1-alf)*100,  "% Confidence Interval: ", range))
    print(paste('True Size: ', true_size))
    print(paste('Is the true value inside the confidence interval: ', inside ))

}

Loading required package: sp

rgeos version: 0.6-2, (SVN revision 693)
 GEOS runtime version: 3.10.2-CAPI-1.16.0 
 Please note that rgeos will be retired during 2023,
plan transition to sf functions using GEOS at your earliest convenience.
 GEOS using OverlayNG
 Linking to sp version: 1.4-7 
 Polygon checking: TRUE 


### Welcome to rworldmap ###

For a short introduction type : 	 vignette('rworldmap')



In [2]:
##############################################
# Importance on uniform sphere
##############################################
n <- 100000
points <- rsphere.normal(n)



latlong <- xyz2latlon(points)

scale_factor <- 510000000/(4*pi)
country_name <- 'China'
true_size <- country_size(country_name)

coords <- coords2country(latlong, country_name)


unif_s_Weight =1/(4*pi) #the density of uniorm sphere
weighted_coord = coords/unif_s_Weight

scaled_mean <- scale_factor*mean(weighted_coord)
scaled_var <- (scale_factor**2) * var(weighted_coord)
alf <- 0.05

print_statements(n, alf, country_name, scaled_mean, scaled_var)


[1] "Country:  China"
[1] "Mean:  9062700"
[1] "Variance:  4539889867608673"
[1] "95 % Confidence Interval:  8645088.04642768"
[2] "95 % Confidence Interval:  9480311.95357232"
[1] "True Size:  9198093"
[1] "Is the true value inside the confidence interval:  TRUE"


In [5]:
# Function for Importance on uniform sphere

Importance_uniform_sphere<-function(n,country_name){
    
    points <- rsphere.normal(n)
    latlong <- xyz2latlon(points)
    
    scale_factor <- 510000000/(4*pi)
    true_size <- country_size(country_name)
    coords <- coords2country(latlong, country_name)
    
    unif_s_Weight =1/(4*pi) #the density of uniorm sphere
    weighted_coord = coords/unif_s_Weight
    
    scaled_mean <- scale_factor*mean(weighted_coord)
    scaled_var <- (scale_factor**2) * var(weighted_coord)
    
    return (c(scaled_mean, scaled_var))
    
}   

In [4]:
########################################
#IMPORTANCE ON RECTANGLE USING BETA DIST.
########################################
#obtain central point of country
# ex for Canada, point is around lat=-100,long=60
#lat in [-180,180], long in [-90,90]

Importance_Beta=function(n,a,country_name){
    # possible to modulate beta parameters to fit properly
    # for ex, with lat=-100,long=60,
    # for long, 360*beta(a=2,b=4.4)-180 distribution approximately has max at -100
    # for long 360*beta(a=6,b=2)-180 distribution approximately has max at 60
    long_lat=country_center(c(country_name))
    
    if (a=="large"){
        a =3
    }
    else if (a=="medium"){
        a =6
    }
    else if(a=="small"){
        a =9
    }
    else if(a=="very small"){
        a =12
    }

    c1=long_lat[1,1] #long
    c2=long_lat[1,2] #lat

    l_lat=c2-90
    u_lat=c2+90

    l_long=c1-180
    u_long=c1+180


    long=360*rbeta(n,a,a)+c1-180
    lat=180*rbeta(n,a,a)+c2-90

    long_lat=matrix(c(long,lat),n,2)
    coords <- coords2country_wrap(long_lat, country_name)
    density_lat=density_beta(lat,a,a,l_lat,u_lat)
    density_long=density_beta(long,a,a,l_long,u_long)
    mat_lat_long=matrix(c(density_lat,density_long),n,2)

    density_weight <- apply(mat_lat_long, 1, prod )

    weighted_coord = coords/density_weight
    scale_fact=510070000/(64800)
    scaled_mean <- scale_fact*mean(weighted_coord)
    scaled_var <- (scale_fact**2) * var(weighted_coord)
    
    return(c(scaled_mean,scaled_var))

}

In [3]:
## Von Mises Fisher
Von_Mises_Fisher<-function(n,concentration, country_name){
    # concentration can be a number, in which case, the following wont evaluate
    # concentration can also be a string, in which case the following will evaluate and set it to the corresponding number
    if (concentration=="large"){
        concentration =3
    }
    else if (concentration=="medium"){
        concentration =5
    }
    else if(concentration=="small"){
        concentration =7
    }
    else if (concentration =="very small"){
        concentration =9
    
    }

    long_lat=country_center(c(country_name))
    long=long_lat[1,1]
    lat=long_lat[1,2]

    lat_rad=deg2rad (lat)
    long_rad=deg2rad(long)
    
    xyz=c(cos(lat_rad)*cos(long_rad),cos(lat_rad)*sin(long_rad),sin(lat_rad))
    points <- rvMF(n,concentration*xyz)
    #print(concentration*xyz)
    #print(xyz)
    long_lat <- xyz2latlon(points)

    scale_factor <- 510072000/(4*pi)

    coords <- coords2country(long_lat, country_name)
    #print(points)
    constant=concentration/((2*pi)*exp((concentration)-exp(-concentration))) # constant for von mises Fisher on R^3

    weight_von_mis = constant*(exp(concentration*(points%*%xyz)))
    #print(weight_von_mis)
    weighted_coord = coords/weight_von_mis

    scaled_mean <- scale_factor*mean(weighted_coord)
    scaled_var <- (scale_factor**2)*(var(weighted_coord))

    return (c(scaled_mean, scaled_var))
}

In [2]:
Von_Mises_Fisher_modified<-function(n, country_name){
    # concentration can be a number, in which case, the following wont evaluate
    # concentration can also be a string, in which case the following will evaluate and set it to the corresponding number
    country_size_estimate= Importance_uniform_sphere(n,country_name)
    
    if (country_size_estimate[1]>=1000000){
        concentration =3
    }
    else if (country_size_estimate[1]>=300000){
        concentration =5
    }
    else if(country_size_estimate[1]>=100000){
        concentration =7
    }
    else if(country_size_estimate[1]<100000){
        concentration=9
    
    }

    long_lat=country_center(c(country_name))
    long=long_lat[1,1]
    lat=long_lat[1,2]

    lat_rad=deg2rad (lat)
    long_rad=deg2rad(long)
    
    xyz=c(cos(lat_rad)*cos(long_rad),cos(lat_rad)*sin(long_rad),sin(lat_rad))
    points <- rvMF(n,concentration*xyz)
    #print(concentration*xyz)
    #print(xyz)
    long_lat <- xyz2latlon(points)

    scale_factor <- 510072000/(4*pi)

    coords <- coords2country(long_lat, country_name)
    #print(points)
    constant=concentration/((2*pi)*exp((concentration)-exp(-concentration))) # constant for von mises Fisher on R^3

    weight_von_mis = constant*(exp(concentration*(points%*%xyz)))
    #print(weight_von_mis)
    weighted_coord = coords/weight_von_mis

    scaled_mean <- scale_factor*mean(weighted_coord)
    scaled_var <- (scale_factor**2)*(var(weighted_coord))

    return (c(scaled_mean, scaled_var))
}

In [11]:
n=100000
alf=0.05



countries <- c('Canada','China','Colombia','Egypt','Cuba','Ireland')
concentrations <-c('large','large','medium','medium','small','small')


for (i in 1:6) {
    
    print(paste("Country: ",countries[i]))
    
    out = Von_Mises_Fisher(n,concentrations[i], countries[i])
    
    print('-----------------------------------------------------------------')
    print("Results for Von Mises Fisher distribution") 
    print_statements(n, alf, countries[i], out[1], out[2])
    
    
    out = Importance_Beta(n,concentrations[i],countries[i])
    
    print('-----------------------------------------------------------------')
    print("Results for Importance Beta sampling")
    print_statements(n, alf, countries[i], out[1], out[2])
    
    out=Importance_uniform_sphere(n,countries[i])
    
    print('-----------------------------------------------------------------')
    print("Results for uniform sampling from sphere")
    print_statements(n, alf, countries[i], out[1], out[2])
    
    out = Von_Mises_Fisher_modified(n, countries[i])
    
    print('-----------------------------------------------------------------')
    print("Results for modified Von Mises Fisher Distribution")
    print_statements(n, alf, countries[i], out[1], out[2])
    print("")
    
}

[1] "Country:  Canada"
[1] "-----------------------------------------------------------------"
[1] "Results for Von Mises Fisher distribution"
[1] "Country:  Canada"
[1] "Mean:  9372949.63140442"
[1] "Variance:  743299542093462"
[1] "95 % Confidence Interval:  9203970.84172287"
[2] "95 % Confidence Interval:  9541928.42108597"
[1] "True Size:  9458906"
[1] "Is the true value inside the confidence interval:  TRUE"
[1] "-----------------------------------------------------------------"
[1] "Results for Importance Beta sampling"
[1] "Country:  Canada"
[1] "Mean:  13265442.1441506"
[1] "Variance:  1850360005073276"
[1] "95 % Confidence Interval:  12998830.9507532"
[2] "95 % Confidence Interval:  13532053.3375479"
[1] "True Size:  9458906"
[1] "Is the true value inside the confidence interval:  FALSE"
[1] "-----------------------------------------------------------------"
[1] "Results for uniform sampling from sphere"
[1] "Country:  Canada"
[1] "Mean:  9853200"
[1] "Variance:  4928095730717

In [9]:
# For the very small countries
n=100000
alf=0.05

countries <- c("Lebanon","Jamaica","Singapore")
concentrations <-c("very small",'very small','very small')


for (i in 1:3) {
    
    print(paste("Country: ",countries[i]))
    
    out = Von_Mises_Fisher(n,concentrations[i], countries[i])
    
    print('-----------------------------------------------------------------')
    print("Results for Von Mises Fisher distribution") 
    print_statements(n, alf, countries[i], out[1], out[2])
    
    
    out = Importance_Beta(n,concentrations[i],countries[i])
    
    print('-----------------------------------------------------------------')
    print("Results for Importance Beta sampling")
    print_statements(n, alf, countries[i], out[1], out[2])
    
    out=Importance_uniform_sphere(n,countries[i])
    
    print('-----------------------------------------------------------------')
    print("Results for uniform sampling from sphere")
    print_statements(n, alf, countries[i], out[1], out[2])
    
    out = Von_Mises_Fisher_modified(n, countries[i])
    
    print('-----------------------------------------------------------------')
    print("Results for modified Von Mises Fisher Distribution")
    print_statements(n, alf, countries[i], out[1], out[2])
    print("")
    
}

[1] "Country:  Lebanon"
[1] "-----------------------------------------------------------------"
[1] "Results for Von Mises Fisher distribution"
[1] "Country:  Lebanon"
[1] "Mean:  9069.53293663773"
[1] "Variance:  256971672017.199"
[1] "95 % Confidence Interval:  5927.62910663487"
[2] "95 % Confidence Interval:  12211.4367666406"
[1] "True Size:  10327"
[1] "Is the true value inside the confidence interval:  TRUE"
[1] "-----------------------------------------------------------------"
[1] "Results for Importance Beta sampling"
[1] "Country:  Lebanon"
[1] "Mean:  8183.31882708707"
[1] "Variance:  278963786687.427"
[1] "95 % Confidence Interval:  4909.72964178784"
[2] "95 % Confidence Interval:  11456.9080123863"
[1] "True Size:  10327"
[1] "Is the true value inside the confidence interval:  TRUE"
[1] "-----------------------------------------------------------------"
[1] "Results for uniform sampling from sphere"
[1] "Country:  Lebanon"
[1] "Mean:  10200"
[1] "Variance:  5201947979479.7

From the results above, it is easy to see that between the 3 largest categories, the confidence intervals never overlap, so there is a statistically significant difference between the surface areas for the 3 categories.

This remains true for all 4 categories(large, medium, small, very small). There are some issues with the uniform distribution, which sometimes gives 0 estimate when the smallest countries are considered but this may be because the sample n is only 10k.

* note that the Singapore surface area is not stored in the R world map database, but the von mises fisher does give a confidence interval which includes 728k (usually the CI is from -2.5k to 8.5k)